In [1]:
googlePath="hdfs:///user/swethakolalapudi/PageRank/web-Google.txt"

googleWeblinks=sc.textFile(googlePath).filter(lambda x:"#" not in x).map(lambda x:x.split("\t"))
# This parses the rows and filters out any comments, headers etc


In [2]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    contribs = []
    for url in urls:
        contribs.append((url, rank / num_urls))
    return contribs

In [3]:
links = googleWeblinks.groupByKey().partitionBy(100).cache()

In [4]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0)) # Initialized the ranks to 1 

In [ ]:
for iteration in range(2):
    contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
    ranks = contribs.reduceByKey(lambda x,y:x+y).mapValues(lambda rank: rank * 0.85 + 0.15)
    
for (link,rank) in ranks.sortBy(lambda x:-x[1]).take(10):
    print("%s has rank: %s." % (link, rank))